In [1]:
#!pip install transformers

In [2]:
#!pip install sentence-transformers

In [147]:
#!pip install transformers
#!pip install sentence-transformers
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd
import os
import re
import sys
import glob
import random
import torch.nn as nn
import xml.etree.ElementTree as ET
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import numpy as np
import random
from os import listdir
from os.path import isfile, join
from sklearn import preprocessing
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from transformers import Trainer, TrainingArguments
# from datasets import load_metric
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from torch.utils.data import DataLoader
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForSequenceClassification
import ast
import scipy
from nltk import ngrams
# from argparse import ArgumentParser
# parser = ArgumentParser()
# parser.add_argument("-multi_lb_classi", default=1, type=int, help="whether train a multi-lable classifier or individual classifiers for each class")
# opt = parser.parse_args()
multi_lb_classi = 0


def preprocess(example_sent):
    global stop_words
    word_tokens = word_tokenize(str(example_sent).lower())
    filtered_sentence = [w for w in word_tokens if not w in stop_words and w.isalpha()]
    new = " " 
    a = new.join(filtered_sentence)
    return a


In [2]:
def get_ngrams(sent):
    return list(ngrams(sent.split(), 2))

In [126]:
def get_dataset_2018(files, folder):
    pairs_lis = []
    prev_corpus = []
    docs_wise = {}
    
    for z,f in enumerate(files):
        cit_text_lis = []
        ref_text_lis = []
        cit_off_lis = []
        ref_off_lis = []
        new_corpus = []

        try:
            a = folder+f+"/Reference_XML/"+f+".xml"
            tree = ET.parse(a)
            root = tree.getroot()
            final =[]
            total = len(root)
            for a in root:
#                 if a.text!='':
#                     final.append(a.text)
                for b in a:
                    if b.text!='':
                        final.append(b.text)
                    
            print(final)            
            d={'col1':final}
            rp = pd.DataFrame(data=d)
            corpus = rp.col1
            new_corpus = corpus.apply(lambda x: preprocess(x))

            data = None
            ann = None
            a_folder = folder+f+"/annotation/"
            file = os.listdir(a_folder)[0]

            ann = a_folder+file
            with open(ann,"r") as file:
                data = file.read()

                cit_text = re.findall("Citation Text:\s+([^|]*)", data)
                pattern = r'\<.*?\>'
                pattern2 = r'\(.*?\)'
                for c in cit_text:
                    c = re.sub(pattern2,'',re.sub(pattern, '', c))
                    c = preprocess(c)
                    cit_text_lis.append(c)


                ref_text = re.findall("Reference Text:\s+([^|]*)", data)
                pattern = r'\<.*?\>'
                pattern2 = r'\(.*?\)'
                for ref in ref_text:
                    ref = re.sub(pattern2,'',re.sub(pattern, '', ref))
                    ref = preprocess(ref)
                    ref_text_lis.append(ref)


    #                 cit_off = re.findall("Citation Offset:\s+([^|]*)", data)
    #                 for c in cit_off:
    #                     c = ast.literal_eval(c)
    #                     cit_off_lis.append(c)


                ref_off = re.findall("Reference Offset:\s+([^|]*)", data)
                for r in ref_off:
                    r = ast.literal_eval(r)
                    ref_off_lis.append(r)

            
#             new_corpus = np.array(new_corpus)
#             updated_corpus = []
#             for sent in new_corpus:
#                 if sent!='':
#                     updated_corpus.append(sent)
                    
          
#             new_corpus = updated_corpus
#             del updated_corpus
            for i in range(len(cit_text_lis)):
                temp = cit_text_lis[i]
                citant_ngrams = get_ngrams(temp)
    #                 reff_ = [int(i) for i in ref_off_lis[i]]
    #                 cited_text_spans = new_corpus[reff_]
    #                 cited_text_spans = pd.Series(cited_text_spans)
    #                 cited_text_ngrams = cited_text_spans.apply(lambda x: get_ngrams(x))
    #                 cited_text_ngrams_lis = []
    #                 for i in range(len(cited_text_ngrams)):
    #                     cited_text_ngrams_lis.extend(cited_text_ngrams[i])
                for j in ref_off_lis[i]:
                    j = int(j)
                    if j< len(new_corpus):
                        cited_text_spans = new_corpus[j]
                        cited_text_ngrams =  get_ngrams(cited_text_spans)
                        lis = np.intersect1d(list(citant_ngrams), list(cited_text_ngrams))
                        lisone = np.intersect1d(temp, list(cited_text_spans))
                        lis = " ".join(lis) + " " + " ".join(lisone)
                        pairs_lis.append(InputExample(texts=[cit_text_lis[i]+" sey "+lis,new_corpus[int(j)]],label=1.0)) #positive pairs
                        pairs_lis.append(InputExample(texts=[cit_text_lis[i]+" sey "+lis,new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.3)) #negative pairs
                        pairs_lis.append(InputExample(texts=[cit_text_lis[i]+" sey "+lis,new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.3))
                        pairs_lis.append(InputExample(texts=[cit_text_lis[i]+" sey "+lis,new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.3))
    #                         pairs_lis.append(InputExample(texts=[new_corpus[random.randint(0,len(new_corpus)-1)],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.3))
    #                         pairs_lis.append(InputExample(texts=[new_corpus[random.randint(0,len(new_corpus)-1)],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.3))

            if (z!=0 and len(new_corpus)!=0 and len(prev_corpus)!=0):
                pairs_lis.append(InputExample(texts = [prev_corpus[random.randint(0,len(prev_corpus)-1)],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.0))
    #                 pairs_lis.append(InputExample(texts = [prev_corpus[random.randint(0,len(prev_corpus)-1)],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.0))
    #                 pairs_lis.append(InputExample(texts = [prev_corpus[random.randint(0,len(prev_corpus)-1)],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.0))
    #                 pairs_lis.append(InputExample(texts = [prev_corpus[random.randint(0,len(prev_corpus)-1)],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.0))

            docs_wise[f] = {'corpus':new_corpus,  'cite_text':cit_text_lis, 'ref_off':ref_off_lis}
            prev_corpus = new_corpus
        except Exception as e: 
            print(f,e)
#             exc_type, exc_obj, exc_tb = sys.exc_info()
#             fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
#             print(exc_type, fname, exc_tb.tb_lineno)
#             prev_corpus = []
    return pairs_lis, docs_wise


In [127]:
docs = os.listdir("/ssd_scratch/cvit/dhawals1939/scisumm-2018/Training")

# random.shuffle(docs)
# train_e = int(docs)
# val_e = int(0.1*len(docs)) + train_e
train_rps = docs
# val_rps = docs[train_e:val_e]
# test_rps = docs[val_e:]

In [129]:
# test_docs  = os.listdir("./scisumm-2018/Test")
stop_words = set(stopwords.words('english'))
train_data, train_docs = get_dataset_2018(train_rps,"/ssd_scratch/cvit/dhawals1939/scisumm-2018/Training/")
# val_data, val_docs = get_dataset_2018(val_rps,"scisumm-2018/Training/")
# test_data, test_docs = get_dataset_2018(test_rps,"/ssd_scratch/cvit/dhawals1939/scisumm-2018/Training/")


['A word in one language can be translated to zero, one, or several words in other languages.', 'Using word fertility features has been shown to be useful in building word alignment models for statistical machine translation.', 'We built a fertility hidden Markov model by adding fertility to the hidden Markov model.', 'This model not only achieves lower alignment error rate than the hidden Markov model, but also runs faster.', 'It is similar in some ways to IBM Model 4, but is much easier to understand.', 'We use Gibbs sampling for parameter estimation, which is more principled than the neighborhood method used in IBM Model 4.', 'IBM models and the hidden Markov model (HMM) for word alignment are the most influential statistical word alignment models (Brown et al., 1993; Vogel et al., 1996; Och and Ney, 2003).', 'There are three kinds of important information for word alignment models: lexicality, locality and fertility.', 'IBM Model 1 uses only lexical information; IBM Model 2 and the

['In this paper, we offer broad insight into the underperformance of Arabic constituency parsing by analyzing the interplay of linguistic phenomena, annotation choices, and model design.', 'First, we identify sources of syntactic ambiguity understudied in the existing parsing literature.', 'Second, we show that although the Penn Arabic Treebank is similar to other tree- banks in gross statistical terms, annotation consistency remains problematic.', 'Third, we develop a human interpretable grammar that is competitive with a latent variable PCFG.', 'Fourth, we show how to build better models for three different parsers.', 'Finally, we show that in application settings, the absence of gold segmentation lowers parsing performance by 2–5% F1.', 'It is well-known that constituency parsing models designed for English often do not generalize easily to other languages and treebanks.1 Explanations for this phenomenon have included the relative informativeness of lexicalization (Dubey and Keller,

J00-3003 not well-formed (invalid token): line 12, column 25
H05-1115 not well-formed (invalid token): line 6, column 109
['New words such as names, technical terms, etc appear frequently.', 'As such, the bilingual lexicon of a machine translation system has to be constantly updated with these new word translations.', 'Comparable corpora such as news documents of the same period from different news agencies are readily available.', 'In this paper, we present a new approach to mining new word translations from comparable corpora, by using context information to complement transliteration information.', 'We evaluated our approach on six months of Chinese and English Gigaword corpora, with encouraging results.', 'New words such as person names, organization names, technical terms, etc. appear frequently.', 'In order for a machine translation system to translate these new words correctly, its bilingual lexicon needs to be constantly updated with new word translations.', 'Much research has 

['The extensive use of Multiword Expressions (MWE) in natural language texts prompts more detailed studies that aim for a more adequate treatment of these expressions.', 'A MWE typically expresses concepts and ideas that usually cannot be expressed by a single word.', 'Intuitively, with the appropriate treatment of MWEs, the results of an Information Retrieval (IR) system could be improved.', 'The aim of this paper is to apply techniques for the automatic extraction of MWEs from corpora to index them as a single unit.', 'Experimental results show improvements on the retrieval of relevant documents when identifying MWEs and treating them as a single indexing unit.', 'One of the motivations of this work is to investigate if the identification and appropriate treatment of Multiword Expressions (MWEs) in an application contributes to improve results and ultimately lead to more precise man-machine interaction.', 'The term “multiword expression” has been used to describe a large set of disti

X96-1048 not well-formed (invalid token): line 14, column 117
['A corpus of German newspaper commentaries has been assembled and annotated with different information (and currently, to different degrees): part-of-speech, syntax, rhetorical structure, connectives, co-reference, and information structure.', 'The paper explains the design decisions taken in the annotations, and describes a number of applications using this corpus with its multi-layer annotation.', 'A corpus of German newspaper commentaries has been assembled at Potsdam University, and annotated with different linguistic information, to different degrees.', 'Two aspects of the corpus have been presented in previous papers ((Re- itter, Stede 2003) on underspecified rhetorical structure; (Stede 2003) on the perspective of knowledge-based summarization).', 'This paper, however, provides a comprehensive overview of the data collection effort and its current state.', 'At present, the ‘Potsdam Commentary Corpus’ (henceforth ‘PCC

['In this paper we specifically address questions of polysemy with respect to verbs, and how regular extensions of meaning can be achieved through the adjunction of particular syntactic phrases.', 'We see verb classes as the key to making gen\xad eralizations about regular extensions of mean\xad ing.', 'Current approaches to English classifica\xad tion, Levin classes and WordNet, have limita\xad tions in their applicability that impede their utility as general classification schemes.', 'We present a refinement of Levin classes, intersec\xad tive sets, which are a more fine-grained clas\xad sification and have more coherent sets of syn\xad tactic frames and associated semantic compo\xad nents.', 'We have preliminary indications that the membership of our intersective sets will be more compatible with WordNet than the orig\xad inal Levin classes.', 'We also have begun to ex\xad amine related classes in Portuguese, and find that these verbs demonstrate similarly coherent syntactic and sem

['Two classes of methods have been shown to be useful for resolving lexical ambiguity.', 'The first relies on the presence of particular words within some distance of the ambiguous target word; the second uses the pattern of words and part-of-speech tags around the target word.', 'These methods have complementary coverage: the former captures the lexical "atmosphere" (discourse topic, tense, etc.), while the latter captures local syntax.', 'Yarowsky has exploited this complementarity by combining the two methods using decision lists.', 'The idea is to pool the evidence provided by the component methods, and to then solve a target problem by applying the single strongest piece of evidence, whatever type it happens to be.', "This paper takes Yarowsky's work as a starting point, applying decision lists to the problem of context-sensitive spelling correction.", 'Decision lists are found, by and large, to outperform either component method.', 'However, it is found that further improvements 

In [130]:
def test_if_valid_data(docs):
    for k in docs:
        try:
            assert(len(docs[k]['ref_off']) == len(docs[k]['cite_text']))
        except:
            print("test failed: length of queries and gt not equal")
            return
    print("unit test passed")
test_if_valid_data(train_docs)

unit test passed


In [131]:
len(train_data), #len(val_data),len(test_data), len(train_docs), len(val_docs),len(test_docs)

(3768,)

In [132]:
#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=16)
# val_dataloader = DataLoader(val_data, shuffle=True, batch_size=1)
# test_dataloader = DataLoader(test_data, shuffle=True, batch_size=1)
                

In [25]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

#Define the model. Either from scratch of by loading a pre-trained model
model = SentenceTransformer('bert-base-nli-mean-tokens')
train_loss = losses.CosineSimilarityLoss(model)
#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=3, warmup_steps=100)


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/229 [00:00<?, ?it/s]

Iteration:   0%|          | 0/229 [00:00<?, ?it/s]

Iteration:   0%|          | 0/229 [00:00<?, ?it/s]

In [26]:
# torch.save(model,'bert2018_ngrams_full_dset.pth')
model = torch.load('bert2018_ngrams_full_dset.pth')

In [28]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [133]:
import pickle
def get_embed_docs(docs):
    for doc_id in docs.keys():
        docs[doc_id]['corpus_embed'] = model.encode(docs[doc_id]['corpus'])
        docs[doc_id]['cite_text_embed'] = model.encode(docs[doc_id]['cite_text'])
    return docs
train_docs = get_embed_docs(train_docs)

with open('train_docs.pkl', 'wb') as handle:
    pickle.dump(train_docs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
# from ignite.metrics import Rouge  #https://pypi.org/project/pytorch-ignite/
from rouge_score import rouge_scorer  #https://pypi.org/project/rouge-score/
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
from pythonrouge.pythonrouge import Pythonrouge


def get_matching_sentences(model,corpus,queries,gt, thresh=0.6,topk=3,cs_based=True):
    #f1 score will be same as precision and recall in our case: since documents
    # Get a vector for each headline (sentence) in the corpus
    # Define search queries and embed them to vectors as well
    
    query_embeddings = model.encode(queries)
    # For each search term return 3 closest sentences
    total_nums_correct = 0
    total_retrieved = 0
    total_relevent = 0
    rouge1 = []
    rouge2 = []
    rouge_su4 = []
    for i in range(len(queries)):
        query = queries[i]
        distances = []
#         lis = np.intersect1d(list(citant_ngrams), list(cited_text_ngrams_lis))
#         lis = " ".join(lis)
#         temp = pd.Series(cit_text_lis[i])
#         citant_ngrams = temp.apply(lambda x: get_ngrams(x))
#         lis = np.intersect1d(list(citant_ngrams), list(cited_text_ngrams_lis))
#         lis = " ".join(lis)
        citant_ngrams = get_ngrams(query)
        for c_sente in corpus:
            rp_ngrams = get_ngrams(c_sente)
            lis = np.intersect1d(list(citant_ngrams), list(rp_ngrams))
            lis = " ".join(lis)
            query_embedding = model.encode(query+lis)
            rp_embedding = model.encode(c_sente)
            distances.append(scipy.spatial.distance.cdist([query_embedding], [rp_embedding], "cosine")[0])
#         query, query_embedding  = queries[i], query_embeddings[i]
#         distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]
        distances = np.array(distances)
        distances = 1 - distances

        results = zip(range(len(distances)), distances)
        results = sorted(results, key=lambda x: -x[1])

        retrieved = []
        if cs_based:
            for k,dist in results:
                if dist >= thresh:
                    retrieved.append(k)
        else: #retrieve topk most matching sentences
            indexes = results[0:topk]
            retrieved = []
            for l,k in indexes:
                retrieved.append(l)
    
        nums_correct = len(np.intersect1d(retrieved,gt[i]))
        total_nums_correct += nums_correct
        total_relevent += len(gt[i])
        total_retrieved += len(retrieved)
        for idx in retrieved:
            scores = scorer.score(corpus[idx].strip(), query)
            rouge = Pythonrouge(summary_file_exist=False,
                    summary=[[corpus[idx].strip()]], reference=[[[query]]],
                    n_gram=2, ROUGE_SU4=True, ROUGE_L=False,
                    recall_only=False, stemming=True, stopwords=True,
                    word_level=True, length_limit=True, length=50,
                    use_cf=False, cf=95, scoring_formula='average',
                    resampling=True, samples=1000, favor=True, p=0.5)
    
            score = rouge.calc_score()
            rouge1.append(score['ROUGE-1-F'])
            rouge2.append(score['ROUGE-2-F'])
            rouge_su4.append(score['ROUGE-SU4-F'])

            
    return total_nums_correct, total_relevent,total_retrieved, np.mean(rouge1), np.mean(rouge2), np.mean(rouge_su4)

In [143]:
doc_id = 'W08-2222'
print(train_docs[doc_id].keys())
print(train_docs[doc_id]['cite_text'][0])
print(train_docs[doc_id]['ref_off'][0])


dict_keys(['corpus', 'cite_text', 'ref_off', 'corpus_embed', 'cite_text_embed'])
use boxer semantic analyzer extract semantic predicates event date
['3', '144']


In [145]:
train_docs[doc_id]['corpus'][3]

'resulting drss translated ordinary logic formulas processing standard theorem provers logic'

In [146]:
train_docs[doc_id]['corpus'][2]

'semantic representations produced boxer known discourse representation structures drss incorporate neodavidsonian representations events using verbnet inventory thematic roles'

In [333]:
path='/ssd_scratch/cvit/dhawals1939/scisumm-2018/Test-Gold/Task1/'
files = [f for f in listdir(path) if isfile(join(path, f))]

def ref_off(d, path='/ssd_scratch/cvit/dhawals1939/scisumm-2018/Test-Gold/Task1/'):
    ref_off_union = {}
    for name in names:
        f_n = d+'_'+name+'.csv'
        if f_n in files:
            df = pd.read_csv(path+f_n)
            ref_offs = list(df['Reference Offset'])
            if(df['Reference Offset'].dtype!=np.float64):
                for i,r in enumerate(ref_offs):
                    r = str(r).replace("'","")
                    if ',' in r:
                        r = r.split(',')
                        for a in r:
                            try:
                                if i in ref_off_union:
                                    ref_off_union[i].add(int(a))
                                else:
                                    ref_off_union[i] = {int(a)}
                            except:
                                pass
                    else:
                        try:
                            if i in ref_off_union:
                                ref_off_union[i].add(int(r))
                            else:
                                ref_off_union[i] = {int(r)}
                        except:
                            pass
            else:
                for i,r in enumerate(ref_offs):
                    if i in ref_off_union:
                        try:
                            ref_off_union[i].add(int(r))
                        except:
                            pass
                    else:
                        try:
                            ref_off_union[i] = {int(r)}
                        except:
                            pass
    print(ref_off_union)
    return ref_off_union            
#     return list(pd.Series(ref_off_union))

def get_Test_dataset_2018(files, folder):
    pairs_lis = []
    prev_corpus = []
    docs_wise = {}
    titles = []
    for z,f in enumerate(files):
        print("\n\nfile ",f)
        cit_text_lis = []
        ref_text_lis = []
        cit_off_lis = []
#         ref_off_lis = []
        new_corpus = []

        a = folder+f+"/Reference_XML/"+f+".xml"
        tree = ET.parse(a)
        root = tree.getroot()
        final =[]
        total = len(root)
        
        for a in root:
            title = a.text
            titles.append(title)
            break
            
        final.append(title)   
        for a in root:
            for b in a:
                final.append(b.text)
                
        d={'col1':final}
        rp = pd.DataFrame(data=d)
        corpus = rp.col1
        new_corpus = corpus.apply(lambda x: preprocess(x))

        path = '/ssd_scratch/cvit/dhawals1939/scisumm-2018/Test-Gold/Task1/'
        files = [f_ for f_ in listdir(path) if isfile(join(path, f_))]
        
        fi = None
        df = None
        for name in names:
            fi = f+'_'+name+'.csv'
            if fi in files:
                df = pd.read_csv(path+fi)
                break
        
        cit_text = list(df['Citation Text Clean'])
        pattern = r'\<.*?\>'
        pattern2 = r'\(.*?\)'
        for c in cit_text:
            c = re.sub(pattern2,'',re.sub(pattern, '', c))
            c = preprocess(c)
            cit_text_lis.append(c)


        ref_text = list(df['Reference Text'])
        pattern = r'\<.*?\>'
        pattern2 = r'\(.*?\)'

        for ref in ref_text:
            try:
                ref = re.sub(pattern2,'',re.sub(pattern, '', ref))
            except:
                print("ref ",ref)
            ref = preprocess(ref)
            ref_text_lis.append(ref)

        ref_off_lis = ref_off(f)
        print("len new_corpus",len(new_corpus))
        print(len(ref_off_lis),len(cit_text_lis))
        for i in range(len(cit_text_lis)):
            if i in ref_off_lis:
                for j in ref_off_lis[i]:
                    cited_text_spans = new_corpus[int(j)]
                    if int(j) < len(new_corpus):
                        pairs_lis.append(InputExample(texts=[cit_text_lis[i],new_corpus[int(j)]],label=1.0)) #positive pairs
                        pairs_lis.append(InputExample(texts=[cit_text_lis[i],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.3)) #negative pairs
                        pairs_lis.append(InputExample(texts=[cit_text_lis[i],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.3))
                        pairs_lis.append(InputExample(texts=[cit_text_lis[i],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.3))
    #                         pairs_lis.append(InputExample(texts=[new_corpus[random.randint(0,len(new_corpus)-1)],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.3))
    #                         pairs_lis.append(InputExample(texts=[new_corpus[random.randint(0,len(new_corpus)-1)],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.3))
            else:
                print("no annotation found for ",i)
        if (z!=0 and len(new_corpus)!=0 and len(prev_corpus)!=0):
            pairs_lis.append(InputExample(texts = [prev_corpus[random.randint(0,len(prev_corpus)-1)],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.0))
#                 pairs_lis.append(InputExample(texts = [prev_corpus[random.randint(0,len(prev_corpus)-1)],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.0))
#                 pairs_lis.append(InputExample(texts = [prev_corpus[random.randint(0,len(prev_corpus)-1)],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.0))
#                 pairs_lis.append(InputExample(texts = [prev_corpus[random.randint(0,len(prev_corpus)-1)],new_corpus[random.randint(0,len(new_corpus)-1)]],label=0.0))

        docs_wise[f] = {'corpus':new_corpus.values,  'cite_text':cit_text_lis, 'ref_off':ref_off_lis}
        prev_corpus = new_corpus

            
    return pairs_lis, docs_wise

test_path = "/ssd_scratch/cvit/dhawals1939/scisumm-2018/Test/"
test_rps = os.listdir(test_path)
stop_words = set(stopwords.words('english'))
test_data, test_docs = get_Test_dataset_2018(test_rps,test_path)




file  A00-2018
{0: {17, 90, 91, 5}, 1: {5}, 2: {17, 90, 91}, 3: {39, 48, 49, 51, 120}, 4: {119, 90, 91, 92, 93, 94}, 5: {0, 162, 95}, 6: {175, 90, 87}, 7: {90, 91, 92, 93, 94}, 8: {1, 38, 39, 40, 176}, 9: {126, 174}, 10: {101, 12, 85}, 11: {174, 143, 146, 155, 63}, 12: {17, 63, 78, 79}, 13: {162, 90, 78}, 14: {40, 91, 174}, 15: {180, 79}, 16: {40}, 17: {175}}
len new_corpus 191
18 15


file  P05-1013
ref  nan
{0: {49, 20}, 1: {24, 9, 109}, 2: {104, 106, 95}, 3: {104, 86, 79}, 4: {20, 109}, 5: {36, 38, 95}, 6: {20, 109}, 7: {36, 79}, 8: {109, 62, 23}, 9: {24, 104, 109}, 10: {80, 81, 109, 23}, 11: {24, 104}, 12: {20, 86, 95}, 13: {24, 96, 95}, 14: {20, 109, 95}, 15: {40, 49, 51}, 16: {99, 109, 7}, 17: {14, 7}, 19: {104, 2, 20}, 18: {49}}
len new_corpus 114
20 20


file  D10-1044
ref  nan
ref  nan
{0: {144, 9, 4}, 1: {96, 132, 95}, 2: {9, 7}, 3: {62}, 4: {50, 28, 71}, 5: {96, 152, 23}, 6: {144, 71}, 8: {9, 75, 62}, 9: {28, 45}, 11: {75}, 12: {143, 31}, 13: {153, 22}, 14: {96, 144, 23}, 

In [334]:
path = '/ssd_scratch/cvit/dhawals1939/scisumm-2018/Test-Gold/Task1/'
files = [f for f in listdir(path) if isfile(join(path, f))]
sorted(files)
lis_missing = []
files_missing = []
doc_ids = []
for f in files:
    df = pd.read_csv(path+f)
    if df['Reference Offset'].isnull().values.any():
#         print("f ",f)
        lis_missing.append('_'+f.split('_')[1])
        files_missing.append(f)
        doc_ids.append(f.split('_')[0])


In [195]:
doc_ids = set(doc_ids)

In [184]:
lis_missing
names, counts = np.unique(lis_missing,return_counts=True)

In [187]:
list(zip(names,counts))

[('_aakansha.csv', 8),
 ('_swastika.csv', 12),
 ('_sweta.csv', 5),
 ('_vardha.csv', 5)]

In [306]:
def ref_off(d, path='/ssd_scratch/cvit/dhawals1939/scisumm-2018/Test-Gold/Task1/'):
    ref_off_union = {}
    for name in names:
        f_n = d+'_'+name+'.csv'
        if f_n in files:
            df = pd.read_csv(path+f_n)
            ref_offs = list(df['Reference Offset'])
            if(df['Reference Offset'].dtype!=np.float64):
                for i,r in enumerate(ref_offs):
                    r = str(r).replace("'","")
                    if ',' in r:
                        r = r.split(',')
                        for a in r:
                            try:
                                if i in ref_off_union:
                                    ref_off_union[i].add(int(a))
                                else:
                                    ref_off_union[i] = {int(a)}
                            except:
                                pass
                    else:
                        try:
                            if i in ref_off_union:
                                ref_off_union[i].add(int(r))
                            else:
                                ref_off_union[i] = {int(r)}
                        except:
                            pass
            else:
                for i,r in enumerate(ref_offs):
                    if i in ref_off_union:
                        try:
                            ref_off_union[i].add(int(r))
                        except:
                            pass
                    else:
                        try:
                            ref_off_union[i] = {int(r)}
                        except:
                            pass
                    
    return list(pd.Series(ref_off_union))
    

In [308]:
path = '/ssd_scratch/cvit/dhawals1939/scisumm-2018/Test-Gold/Task1/'
names = ['aakansha','vardha','swastika','sweta']

ref_off_dic = {}
for d in doc_ids:
    ref_off_dic[d] = ref_off(d, path)

In [309]:
ref_off_dic

{'P05-1013': [{20, 49},
  {9, 24, 109},
  {95, 104, 106},
  {79, 86, 104},
  {20, 109},
  {36, 38, 95},
  {20, 109},
  {36, 79},
  {23, 62, 109},
  {24, 104, 109},
  {23, 80, 81, 109},
  {24, 104},
  {20, 86, 95},
  {24, 95, 96},
  {20, 95, 109},
  {40, 49, 51},
  {7, 99, 109},
  {7, 14},
  {2, 20, 104},
  {49}],
 'J01-2004': [{372},
  {15, 17, 40, 41, 42},
  {21, 25, 31},
  {21, 31, 364},
  {31, 302},
  {31},
  {79, 80, 215, 231},
  {21, 297, 302},
  {133, 138},
  {291},
  {355, 372},
  {31, 59, 209, 210},
  {100, 372},
  {20, 21, 32, 108},
  {31},
  {21, 31, 33}],
 'P08-1043': [{94, 156},
  {4},
  {19, 70, 94},
  {3, 19},
  {4, 19, 51},
  {4, 19, 107},
  {14, 69, 70, 105},
  {33, 85},
  {19, 53},
  {19, 48},
  {19, 141, 163},
  {21, 100, 188},
  {94},
  {133, 134, 188},
  {85, 86, 156},
  {5, 97, 155}],
 'A00-2030': [{4, 11, 18},
  {33, 34, 52, 100},
  {49, 50, 52},
  {33, 34},
  {1, 10, 106},
  {6, 16, 61},
  {6, 33, 34, 104},
  {32, 33, 34},
  {2, 12, 23, 24},
  {3, 23, 24, 61},
  

In [ ]:
path = '/ssd_scratch/cvit/dhawals1939/scisumm-2018/Test-Gold/Task1/'
names = ['aakansha','vardha','swastika','sweta']


for d in doc_ids:
    df = pd.read_csv(path+)

In [165]:
path = '/ssd_scratch/cvit/dhawals1939/scisumm-2018/Test-Gold/Task1/'

f = 'P05-1013_swastika.csv'
df = pd.read_csv(path+f)
df

,Citance Number,Reference Article,Citing Article,Citation Marker Offset,Citation Marker,Citation Offset,Citation Text,Citation Text Clean,Reference Offset,Reference Text,Discourse Facet
0,1,P05-1013,W05-1505,0,"Nivre and Nilsson, 2005",0,Recent work by Nivre and Nilsson introduces a ...,Recent work by Nivre and Nilsson introduces a ...,49.0,"<S sid=""49"" ssid=""20"">The baseline simply reta...",Method_Citation
1,2,P05-1013,P08-1006,0,"Nivre and Nilsson, 2005",0,1http: //sourceforge.net/projects/mstparser Fi...,"Sagae and Tsujii (2007)'s dependency parser, b...",109.0,"<S sid=""109"" ssid=""1"">We have presented a new ...",Method_Citation
2,3,P05-1013,W10-1401,0,"Nivre and Nilsson, 2005",0,Bengoetxea and Gojenola (2010) discuss non-pro...,Bengoetxea and Gojenola (2010) discuss non-pro...,95.0,"<S sid=""95"" ssid=""6"">The second main result is...",Result_Citation
3,4,P05-1013,P12-3029,0,"Nivre and Nilsson, 2005",0,For tree banks with non-projective trees weuse...,For tree banks with non-projective trees we us...,79.0,"<S sid=""79"" ssid=""6"">In the first part of the ...",Method_Citation
4,5,P05-1013,W10-1403,0,"Nivre and Nilsson, 2005",0,It uses graph transformation to handle non-pro...,It uses graph transformation to handle non-pro...,109.0,"<S sid=""109"" ssid=""1"">We have presented a new ...",Result_Citation
5,6,P05-1013,D08-1008,0,"Nivre and Nilsson, 2005",0,"To simplify implementation, we instead opted f...","To simplify implementation, we instead opted f...",38.0,"<S sid=""38"" ssid=""9"">Projectivizing a dependen...",Method_Citation
6,7,P05-1013,D07-1013,0,"Nivre and Nilsson, 2005",0,",wn in O (n) time, producing a projective depe...",Nivre and Nilsson (2005) showed how the restri...,109.0,"<S sid=""109"" ssid=""1"">We have presented a new ...",Result_Citation
7,8,P05-1013,D07-1119,0,"Nivre and Nilsson, 2005",0,"For handling non-projective relations, Nivre a...","For handling non-projective relations, Nivre a...",79.0,"<S sid=""79"" ssid=""6"">In the first part of the ...",Method_Citation
8,9,P05-1013,N07-1050,0,"Nivre and Nilsson, 2005",0,Whereas most of the early approaches were limi...,The most popular strategy for capturing non pr...,109.0,"<S sid=""109"" ssid=""1"">We have presented a new ...",Result_Citation
9,10,P05-1013,W09-1207,0,"Nivre and Nilsson, 2005",0,"troduced in (Nivre and Nilsson, 2005) to handl...",We adopt the pseudo-projective approach introd...,109.0,"S sid=""109"" ssid=""1"">We have presented a new m...",Result_Citation


In [153]:
df = pd.read_csv('/ssd_scratch/cvit/dhawals1939/scisumm-2018/Test-Gold/Task1/A00-2018_aakansha.csv')
df

,Citance Number,Reference Article,Citing Article,Citation Marker Offset,Citation Marker,Citation Offset,Citation Text,Citation Text Clean,Reference Offset,Reference Text,Discourse Facet
0,2,A00-2018,N10-1002,0,"Charniak, 2000",0,"As a benchmark VPC extraction system, we use t...","As a benchmark VPC extraction system, we use t...","'90' , '91'","<S sid=""90"" ssid=""1"">We created a parser based...",Method_Citation
1,3,A00-2018,W11-0610,0,"Charniak, 2000",0,Each of these scores can be calculated from a ...,Each of these scores can be calculated from a ...,'5',"<S sid=""5"" ssid=""1"">We present a new parser fo...",Method_Citation
2,4,A00-2018,W06-3119,0,"Charniak, 2000",0,"We then use Charniak? s parser (Charniak, 2000...","We then use Charniak's parser (Charniak, 2000)...",'90',"<S sid=""90"" ssid=""1"">We created a parser based...",Method_Citation
3,5,A00-2018,N03-2024,0,"Charniak, 2000",0,We were interested in the occurrence of featur...,We were interested in the occurrence of featur...,"'48','49','51'","<S sid=""48"" ssid=""17"">Maximum-entropy models h...",Method_Citation
4,6,A00-2018,N06-1039,0,"Charniak, 2000",0,"After getting a set of basic clusters, we pass...","After getting a set of basic clusters, we pass...","'90','91','92','93','94'","<S sid=""90"" ssid=""1"">We created a parser based...",Method_Citation
5,7,A00-2018,C04-1180,0,2000,0,The levels of accuracy and robustness recently...,The levels of accuracy and robustness recently...,0,“NA”,Result_Citation
6,8,A00-2018,W05-0638,0,"Charniak, 2000",0,"In CoNLL-2005, full parsing trees are provided...","In CoNLL-2005, full parsing trees are provided...",'90',"<S sid=""90"" ssid=""1"">We created a parser based...",Method_Citation
7,9,A00-2018,P05-1065,0,"Charniak, 2000",0,We also use a standard statistical parser (Cha...,We also use a standard statistical parser (Cha...,"'90','91','92','93','94'","<S sid=""90"" ssid=""1"">We created a parser based...",Method_Citation
8,10,A00-2018,P05-1065,0,"Charniak, 2000",0,"For each article, we calculated the per cent a...","For each article, we calculated the percentage...","'38','39','40'","<S sid=""38"" ssid=""7"">To compute a probability ...",Method_Citation
9,11,A00-2018,P04-1040,0,2000,0,The evaluation of the transformed output of th...,The evaluation of the transformed output of th...,'174',"<S sid=""174"" ssid=""1"">We have presented a lexi...",Result_Citation


In [28]:
from os import listdir
from os.path import isfile, join
mypath = '/ssd_scratch/cvit/dhawals1939/scisumm-2018/Test-Gold/Task1/'

In [22]:
onlyfiles

['P87-1015_sweta.csv',
 'P05-1013_swastika.csv',
 'J01-2004_sweta.csv',
 'P08-1043_swastika.csv',
 'P08-1043_sweta.csv',
 'A00-2030_vardha.csv',
 'P05-1013_aakansha.csv',
 'W06-3114_sweta.csv',
 'P11-1061_aakansha.csv',
 'P11-1061_swastika.csv',
 'P11-1061_sweta.csv',
 'J01-2004_aakansha.csv',
 'J01-2004_swastika.csv',
 'W06-2932_swastika.csv',
 'W11-2123_swastika.csv',
 'A00-2030_aakansha.csv',
 'E03-1005_sweta.csv',
 'E03-1005_swastika.csv',
 'A97-1014_sweta.csv',
 'D10-1044_aakansha.csv',
 'W99-0613_swastika.csv',
 'A97-1014_vardha.csv',
 'P08-1102_swastika.csv',
 'W99-0613_vardha.csv',
 'P08-1028_swastika.csv',
 'W06-3114_aakansha.csv',
 'P08-1028_sweta.csv',
 'A00-2030_sweta.csv',
 'W99-0623_sweta.csv',
 'A00-2018_sweta.csv',
 'D10-1044_sweta.csv',
 'W06-3114_swastika.csv',
 'P11-1060_sweta.csv',
 'D10-1044_swastika.csv',
 'A00-2018_aakansha.csv',
 'P04-1036_swastika.csv',
 'W11-2123_aakansha.csv',
 'W06-2932_sweta.csv',
 'W99-0613_sweta.csv',
 'P05-1013_vardha.csv',
 'W99-0623_sw

In [17]:
df = pd.read_csv('/ssd_scratch/cvit/dhawals1939/scisumm-2018/Test-Gold/Task1/P11-1060_aakanksha.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/ssd_scratch/cvit/dhawals1939/scisumm-2018/Test-Gold/Task1/P11-1060_aakanksha.csv'

In [13]:
test_docs['A00-2018']['ref_off']

[]

In [65]:
os.environ['TOKENIZERS_PARALLELISM']= '0'
def evaluate(model, docs, thresh=0.6,topk=3,cs_based=True):
    rouge1_lis = []
    rouge2_lis = []
    rouge_su4_lis = []
    tp_big = 0
    tot_relevent = 0
    tot_retrieved = 0

    for k in docs:
        tp, rele,ret, rouge1, rouge2, rouge_su4 = get_matching_sentences(model,docs[k]['corpus'], docs[k]['cite_text'], docs[k]['ref_off'],thresh,topk,cs_based)
        print("doc_id",k,tp, rele,ret, rouge1, rouge2, rouge_su4)
        
        tp_big += tp
        tot_relevent += rele
        tot_retrieved += ret
        rouge1_lis.append(rouge1)
        rouge2_lis.append(rouge2)
        rouge_su4_lis.append(rouge_su4)
    recall = tp_big/tot_relevent
    precision = tp_big/tot_retrieved
    f1 = 2*recall*precision/(recall+precision+1e-10)
    return recall,precision,f1, np.mean(rouge1_lis), np.mean(rouge2_lis), np.mean(rouge_su4_lis)

In [61]:
train_docs['D10-1058']

{'corpus': array(['word one language translated zero one several words languages',
        'using word fertility features shown useful building word alignment models statistical machine translation',
        'built fertility hidden markov model adding fertility hidden markov model',
        'model achieves lower alignment error rate hidden markov model also runs faster',
        'similar ways ibm model much easier understand',
        'use gibbs sampling parameter estimation principled neighborhood method used ibm model',
        'ibm models hidden markov model hmm word alignment influential statistical word alignment models brown et vogel et och ney',
        'three kinds important information word alignment models lexicality locality fertility',
        'ibm model uses lexical information ibm model hidden markov model take advantage lexical locality information ibm models use three kinds information remain state art despite fact developed almost two decades ago',
        'recent expe

In [59]:
test_docs['A00-2018']

{'corpus': array(['present new parser parsing penn style parse trees achieves average sentences less length less trained tested previously established quot standard quot sections wall street journal treebank',
        'represents decrease error rate best results corpus',
        'major technical innovation use quot quot model conditioning smoothing let us successfully test combine many different conditioning events',
        'also present partial results showing effects different conditioning information including surprising improvement due guessing lexical head guessing lexical head',
        'present new parser parsing penn style parse trees achieves average sentences length sentences length trained tested previously established quot standard quot sections wall street journal',
        'represents decrease error rate best results corpus',
        'following parser based upon probabilistic generative model',
        'sentences parses parser assigns probability p p r equality holding r

In [57]:
recall,precision,f1, rouge1, rouge2, rouge_su4 = evaluate(model,test_docs,cs_based=False,topk=10)   
print("metrics obtained in train: recall {}, precision {}, f1-score {}, rouge1 {}, rouge2 {}, rouge_su4 {}".format(recall,precision,f1, rouge1, rouge2, rouge_su4))

doc_id A00-2018 0 0 0 nan nan nan
doc_id P05-1013 0 0 0 nan nan nan
doc_id D10-1044 0 0 0 nan nan nan
doc_id P87-1015 0 0 0 nan nan nan
doc_id P08-1102 0 0 0 nan nan nan
doc_id W06-3114 0 0 0 nan nan nan
doc_id E03-1005 0 0 0 nan nan nan
doc_id W99-0623 0 0 0 nan nan nan
doc_id P08-1028 0 0 0 nan nan nan
doc_id W99-0613 0 0 0 nan nan nan
doc_id A97-1014 0 0 0 nan nan nan
doc_id P11-1060 0 0 0 nan nan nan
doc_id P11-1061 0 0 0 nan nan nan
doc_id W06-2932 0 0 0 nan nan nan
doc_id A00-2030 0 0 0 nan nan nan
doc_id P08-1043 0 0 0 nan nan nan
doc_id D09-1092 0 0 0 nan nan nan
doc_id J01-2004 0 0 0 nan nan nan
doc_id W11-2123 0 0 0 nan nan nan
doc_id P04-1036 0 0 0 nan nan nan


ZeroDivisionError: division by zero

In [ ]:
recall,precision,f1, rouge1, rouge2, rouge_su4 = evaluate(model,test_docs,thresh=0.65)   
print("metrics obtained in test: recall {}, precision {}, f1-score {}, rouge1 {}, rouge2 {}, rouge_su4 {}".format(recall,precision,f1, rouge1, rouge2, rouge_su4))

In [ ]:
test_docs['C08-1098']

In [ ]:
recall,precision,f1, rouge1, rouge2, rouge_su4 = evaluate(model,test_docs,thresh=0.6)   
print("metrics obtained in test: recall {}, precision {}, f1-score {}, rouge1 {}, rouge2 {}, rouge_su4 {}".format(recall,precision,f1, rouge1, rouge2, rouge_su4))

In [ ]:
recall,precision,f1, rouge1, rouge2, rouge_su4 = evaluate(model,test_docs,thresh=0.55)   
print("metrics obtained in test: recall {}, precision {}, f1-score {}, rouge1 {}, rouge2 {}, rouge_su4 {}".format(recall,precision,f1, rouge1, rouge2, rouge_su4))

In [ ]:
recall,precision,f1, rouge1, rouge2, rouge_su4 = evaluate(model,test_docs,cs_based=False)   
print("metrics obtained in test: recall {}, precision {}, f1-score {}, rouge1 {}, rouge2 {}, rouge_su4 {}".format(recall,precision,f1, rouge1, rouge2, rouge_su4))